┌ Info: Recompiling stale cache file /Users/manvithaponnapati/.julia/compiled/v1.2/JLD2/O1EyT.ji for JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819]
└ @ Base loading.jl:1240


In [25]:
using JLD2, FileIO
using Flux, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated

f = jldopen("../data/mixed_layer_simulation_Q-100_dTdz0.010_tau0.00_profiles.jld2","r+")
timeseries_keys = keys(f["timeseries"])
print(timeseries_keys)


print(f["timeseries"]["w"])
for k in keys(f["timeseries"])
#     println(f["timeseries"][k])
end

#X -> mxn floats
#Y -> Prediction value

y_dim = 1

m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, y_dim),
  softmax) |> gpu

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM()

Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))

accuracy(X, Y)

["t", "vv", "w", "nu", "kappaT", "v", "uv", "ww", "u", "uw", "T", "wT", "uu", "vw"]JLD2.Group
 ├─🔢 0
 ├─🔢 161
 ├─🔢 262
 ├─🔢 335
 ├─🔢 461
 ├─🔢 615
 ├─🔢 762
 ├─🔢 927
 ├─🔢 1099
 ├─🔢 1278
 ├─🔢 1465
 ├─🔢 1664
 ├─🔢 1869
 ├─🔢 2072
 ├─🔢 2282
 ├─🔢 2506
 ├─🔢 2719
 ├─🔢 2949
 ├─🔢 3181
 ├─🔢 3413
 ├─🔢 3650
 ├─🔢 3882
 ├─🔢 4109
 ├─🔢 4333
 ├─🔢 4567
 ├─🔢 4802
 ├─🔢 5040
 ├─🔢 5288
 ├─🔢 5559
 ├─🔢 5813
 ├─🔢 6072
 ├─🔢 6328
 ├─🔢 6600
 ├─🔢 6862
 ├─🔢 7122
 ├─🔢 7375
 ├─🔢 7639
 ├─🔢 7894
 ├─🔢 8150
 ├─🔢 8418
 ├─🔢 8683
 ├─🔢 8953
 ├─🔢 9218
 ├─🔢 9484
 ├─🔢 9746
 ├─🔢 10012
 ├─🔢 10295
 ├─🔢 10574
 ├─🔢 10832
 ├─🔢 11100
 ├─🔢 11377
 ├─🔢 11659
 ├─🔢 11930
 ├─🔢 12199
 ├─🔢 12469
 ├─🔢 12740
 ├─🔢 13012
 ├─🔢 13306
 ├─🔢 13583
 ├─🔢 13879
 ├─🔢 14170
 ├─🔢 14462
 ├─🔢 14750
 ├─🔢 15023
 ├─🔢 15314
 ├─🔢 15597
 ├─🔢 15905
 ├─🔢 16206
 ├─🔢 16502
 ├─🔢 16786
 ├─🔢 17081
 ├─🔢 17376
 ├─🔢 17666
 ├─🔢 17961
 ├─🔢 18256
 ├─🔢 18554
 ├─🔢 18845
 ├─🔢 19140
 ├─🔢 19453
 ├─🔢 19757
 ├─🔢 20081
 ├─🔢 20385
 ├─🔢 20718
 ├─🔢 21047
 ├─🔢 21363
 ├─🔢 21668
 ├─🔢 21985
 ├─🔢

 ├─🔢 443022
 ├─🔢 443491
 ├─🔢 443972
 ├─🔢 444460
 ├─🔢 444960
 ├─🔢 445432
 ├─🔢 445887
 ├─🔢 446372
 ├─🔢 446866
 ├─🔢 447366
 ├─🔢 447893
 ├─🔢 448424
 ├─🔢 448917
 ├─🔢 449345
 ├─🔢 449775
 ├─🔢 450215
 ├─🔢 450675
 ├─🔢 451139
 ├─🔢 451628
 ├─🔢 452098
 ├─🔢 452563
 ├─🔢 453044
 ├─🔢 453514
 ├─🔢 453936
 ├─🔢 454349
 ├─🔢 454778
 ├─🔢 455196
 ├─🔢 455666
 ├─🔢 456170
 ├─🔢 456665
 ├─🔢 457128
 ├─🔢 457569
 ├─🔢 458049
 ├─🔢 458504
 ├─🔢 458971
 ├─🔢 459415
 ├─🔢 459935
 ├─🔢 460486
 ├─🔢 461005
 ├─🔢 461550
 ├─🔢 462079
 ├─🔢 462610
 ├─🔢 463112
 ├─🔢 463601
 ├─🔢 464071
 ├─🔢 464553
 ├─🔢 465107
 ├─🔢 465612
 ├─🔢 466093
 ├─🔢 466571
 ├─🔢 467127
 ├─🔢 467668
 ├─🔢 468165
 ├─🔢 468645
 ├─🔢 469140
 └─🔢 469664

In [23]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated

#X -> mxn floats
#Y -> Prediction value

y_dim = 1

m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, y_dim),
  softmax) |> gpu

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM()

Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))

accuracy(X, Y)

┌ Info: Recompiling stale cache file /Users/manvithaponnapati/.julia/compiled/v1.2/DifferentialEquations/UQdwS.ji for DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1240


retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 34-element Array{Float64,1}:
  0.0                
  0.0776084743154256 
  0.23264513699277584
  0.4291185174543143 
  0.6790821776882875 
  0.9444045910389707 
  1.2674601253261835 
  1.6192913723304114 
  1.9869755337814992 
  2.264090367186479  
  2.5125486490278424 
  2.746828248824617  
  3.038006827022277  
  ⋮                  
  6.455761908474447  
  6.780495751224447  
  7.171039674548505  
  7.584862904164952  
  7.978068388305894  
  8.483164907244102  
  8.719247868929038  
  8.949206527971544  
  9.200184813643565  
  9.438028630962807  
  9.711807852444823  
 10.0                
u: 34-element Array{Array{Float64,1},1}:
 [1.0, 1.0]                               
 [1.0454942346944578, 0.8576684823217127] 
 [1.1758715885138267, 0.639459570317544]  
 [1.4196809607170826, 0.4569962601282084] 
 [1.8767193485546056, 0.32473343696185236]
 [2.5882499852859384, 0.26336255804531]   
 [3.860708771268753, 0

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed Calculus ──────────────────── v0.5.0
 Installed DiffRules ─────────────────── v0.0.10
 Installed DiffEqJump ────────────────── v6.2.2
 Installed Parameters ────────────────── v0.12.0
 Installed DifferentialEquations ─────── v6.8.0
 Installed ArnoldiMethod ─────────────── v0.0.4
 Installed DelayDiffEq ───────────────── v5.17.0
 Installed DiffEqBase ────────────────── v6.4.1
 Installed PoissonRandom ─────────────── v0.4.0
 Installed ForwardDiff ───────────────── v0.10.4
 Installed MultiScaleArrays ──────────── v1.5.0
 Installed RandomNumbers ─────────────── v1.3.0
 Installed MatrixFactorizations ──────── v0.1.0
 Installed GenericSVD ────────────────── v0.2.2
 Installed Sundials ──────────────────── v3.7.0
 Installed IterativeSolvers ──────────── v0.8.1
 Installed CommonSubexpressions ──────── v0.2.0
 Installed LineSearches ───